In [1]:
import geopandas
import pandas as pd
from math import ceil
from balancer import MasterPlan

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [15]:
Hectare = 10000

In [4]:
school = geopandas.read_parquet('../output_data/schools.parquet')
kindergarten = geopandas.read_parquet('../output_data/kindergartens.parquet')
greening = geopandas.read_parquet('../output_data/recreational_areas.parquet')

In [7]:
gdf = geopandas.read_parquet('../output_data/GLEB_BLOCKS_AND_BUILDINGS_INFO_2.parquet')

In [11]:
gdf.head(2)

,geometry,block_id,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area,current_building_area
0,"POLYGON ((316983.653 6644038.696, 316986.485 6...",0,0.0,0.0,0.0000,1091.0,219008.0,237.0,1208.0720,2.738972e+06,1208.0720
1,"POLYGON ((315287.843 6646411.375, 315288.850 6...",1,671.0,1.0,14106.1824,452.0,13070.0,0.0,16625.2319,2.795374e+06,30731.4143


In [9]:
gdf['current_building_area'] = gdf['current_living_area'] + gdf['current_industrial_area']

In [10]:
gdf_ = gdf[['block_id', 'area', 'current_living_area', 'current_industrial_area',
     'current_population', 'current_green_area', 'floors']]

In [12]:
gdf_ = gdf_.merge(school[['id', 'population_unprov_schools']], left_on='block_id', right_on='id').merge(kindergarten[['id', 'population_unprov_kindergartens']], left_on='block_id', right_on='id').merge(greening[['id', 'population_unprov_recreational_areas']], left_on='block_id', right_on='id')

In [13]:
gdf_.drop(['id_x', 'id_y', 'id'], axis=1, inplace=True)

In [16]:
gdf_['area'] = gdf_['area']/Hectare
gdf_['current_living_area'] = gdf_['current_living_area']/Hectare
gdf_['current_industrial_area'] = gdf_['current_industrial_area']/Hectare
gdf_['current_green_area'] = gdf_['current_green_area']/Hectare

In [22]:
# sample = gdf_[gdf_['current_population']!=0].sample()
sample = gdf_[gdf_['area']>10].sample()
sample = sample.to_dict('records')
block = sample[0].copy()
block

{'block_id': 2157,
 'area': 269.91357042657916,
 'current_living_area': 14.3349511,
 'current_industrial_area': 10.115001506,
 'current_population': 28909.0,
 'current_green_area': 24.4459,
 'floors': 0.0,
 'population_unprov_schools': 0,
 'population_unprov_kindergartens': 0,
 'population_unprov_recreational_areas': 0}

In [23]:


mp = MasterPlan(area=block['area'], current_living_area=block['current_living_area'],
                current_industrial_area=block['current_industrial_area'],
                current_population=block['current_population'],
                current_green_area=block['current_green_area'],
                current_unprov_schoolkids=block['population_unprov_schools'],
                current_unprov_kids=block['population_unprov_kindergartens'],
                current_unprov_green_population=block['population_unprov_recreational_areas'])

In [24]:
mp.func()

/home/gk/jupyter/masterplanning/balancer/balancer.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/home/gk/jupyter/masterplanning/balancer/balancer.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/home/gk/jupyter/masterplanning/balancer/balancer.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/home/gk/jupyter/masterplanning/balancer/balancer.py:154: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results

In [25]:
mp.results[['fun', 'x']]

,fun,x
0,240.479207,"[3607.467804926194, 0.003, 0.0006]"
1,231.497184,"[4806.029667247868, 0.003, 0.0006]"
2,180.379970,"[11266.826824031874, 0.003, 0.0009]"
3,130.922326,"[20637.3125, 0.0024000000000000002, 0.0009]"
4,-43.067800,"[39359.67039896645, 0.003, 0.0007684448570410059]"


In [26]:
res = mp.optimal_values(*mp.select_optimal())
res['block_id'] = block['block_id']
res

{'area': 269.91357042657916,
 'population': 49547.0,
 'b': 24.000000000000004,
 'green_coef_G': 9.0,
 'living_area': 63.8661511,
 'school_area': 4.7,
 'school_capacity': 2500,
 'kindergarten_area': 4.4,
 'kindergarten_capacity': 1120,
 'green_area': 43.0201,
 'G_min_capacity': 71700.16666666667,
 'G_max_capacity': 35850.083333333336,
 'green_coef_G_capacity': 47800.11111111111,
 'op_area': 0.061914000000000004,
 'parking1_area': 15.602328000000002,
 'parking2_area': 15.16893,
 'block_id': 2157}

In [27]:
import pickle

with open('../output_data/balanced_result_block.pkl', 'wb') as f:
    pickle.dump(res, f)

In [28]:
block

{'block_id': 2157,
 'area': 269.91357042657916,
 'current_living_area': 14.3349511,
 'current_industrial_area': 10.115001506,
 'current_population': 28909.0,
 'current_green_area': 24.4459,
 'floors': 0.0,
 'population_unprov_schools': 0,
 'population_unprov_kindergartens': 0,
 'population_unprov_recreational_areas': 0}